In [ ]:
import pandas as pd
import numpy as np

P3G_labeling_table = pd.read_csv('P3G_labeling_data.csv')
time_window_start = 0
time_window_end = 21

rows_in_window = (P3G_labeling_table['t_experimental'] >= time_window_start) & \
                 (P3G_labeling_table['t_experimental'] <= time_window_end)

filtered_values = P3G_labeling_table.loc[rows_in_window, 'mean']
median_within_window = filtered_values.median()

In [ ]:
from sympy import symbols, Eq, solve

# Read the specific_rates.csv file
rates_df = pd.read_csv("specific_rates.csv")

# Extract qAc with 2 decimal places
qAc = round(rates_df.loc[rates_df["Variable"] == "qAc", "Value"].values[0], 2)

C13max = (1/6)*100
percent_13C_P3G_plateau = median_within_window
percent_P3G_from_rubisco = (percent_13C_P3G_plateau / C13max) * 100

mol_P3G_NTS, mol_P3G_EMP, percent_EMP = symbols('mol_P3G_NTS mol_P3G_EMP percent_EMP')

eq1 = Eq(mol_P3G_NTS / mol_P3G_EMP, percent_P3G_from_rubisco / (100 - percent_P3G_from_rubisco))
eq2 = Eq(percent_EMP / (100 - percent_EMP), (mol_P3G_EMP * (0.5 * 6)) / (mol_P3G_NTS * 2.5))
eq3 = Eq(mol_P3G_NTS + 2*mol_P3G_EMP, 0.5 * ((-1*qAc) + mol_P3G_NTS + mol_P3G_EMP))

solutions = solve([eq1, eq2, eq3], [mol_P3G_NTS, mol_P3G_EMP, percent_EMP], dict=True)
sol = solutions[0]
mol_P3G_NTS_val = float(sol[mol_P3G_NTS])
mol_P3G_EMP_val = float(sol[mol_P3G_EMP])
percent_EMP_val = float(sol[percent_EMP])

#to be used later in the plotting
y_baseline = 100-percent_EMP_val 

In [ ]:
from cobra.io import load_json_model
import os
import pandas as pd

# Load the JSON model
model = load_json_model('Accumulibacter_anaerobic.json')

# constrain reactions
for rxn_id in ['ADPPPT', 'AMPPPT', 'PYK', 'PDH', 'PHBsyn',
               'SBPase', 'RbuK', 'RbuCO']:
    model.reactions.get_by_id(rxn_id).lower_bound = 0

# Set the objective
model.objective = 'EX_PHB'

# set exchange bounds:
model.reactions.EX_Ace.lower_bound   = qAc
model.reactions.EX_Mal4.lower_bound  = -1000
model.reactions.EX_Mal3.upper_bound  =  1000
model.reactions.EX_PHB.upper_bound   =  1000

# Constrain RbuCO flux exactly to mol_P3G_NTS_val / 2
rbuco = model.reactions.get_by_id('RbuCO')
rbuco.lower_bound = mol_P3G_NTS_val / 2
rbuco.upper_bound = mol_P3G_NTS_val / 2

# Run FBA
solution = model.optimize()

# Extract all fluxes as a dataframe
fluxes_df = solution.fluxes.reset_index()
fluxes_df.columns = ['Reaction', 'Flux']

# Save to CSV
fluxes_df.to_csv('representative_fluxes_labeling.csv', index=False)

pathway_name = 'fluxes_labeling_for_MDF'

# Exchange reactions for record
ex_rxns = ['EX_Ace','EX_CO2','EX_Mal3','EX_Mal4',
           'EX_HB','EX_PHB','EX_H2O','EX_PolyPP','EX_PolyP','EX_PPi','EX_Pi']

# Compute the q‐rates. normalization optional
mal3_flux = solution.fluxes['EX_Mal3']
row = {'pathway': pathway_name}
for ex in ex_rxns:
    flux = solution.fluxes.get(ex, 0.0)
    #row[ex.replace('EX_','')] = flux / mal3_flux
    row[ex.replace('EX_','')] = flux

# CSV filename
summary_file = 'stoichiometry_labeling.csv'

# Read existing or create new
if os.path.exists(summary_file):
    summary = pd.read_csv(summary_file)
    # Overwrite or append the row for this pathway
    summary = summary[summary['pathway'] != pathway_name]
else:
    # Create empty DataFrame with correct columns and dtypes
    cols = ['pathway'] + [ex.replace('EX_','') for ex in ex_rxns]
    dtypes = {col: 'float' for col in cols}
    dtypes['pathway'] = 'str'
    summary = pd.DataFrame({col: pd.Series(dtype=dt) for col, dt in dtypes.items()})

# Append new data
summary = pd.concat([summary, pd.DataFrame([row])], ignore_index=True)

# Save
summary.to_csv(summary_file, index=False)
print(f"Updated {summary_file}")



In [ ]:
import pandas as pd

rxn_df = pd.DataFrame({
    'reaction_id': [r.id for r in model.reactions],
    'equation'  : [r.reaction for r in model.reactions]
})
print(rxn_df)

print(solution.fluxes)

In [ ]:
#Checking the balance of Phosphate

import pandas as pd

pi = model.metabolites.get_by_id('Pi')

# Build a dict of {reaction_id: flux}
pi_fluxes = {}
for rxn in pi.reactions:
    # use .get() to default to 0.0 if somehow missing
    pi_fluxes[rxn.id] = solution.fluxes.get(rxn.id, 0.0)


df_pi = pd.DataFrame.from_dict(pi_fluxes, orient='index', columns=['Flux'])
df_pi.index.name = 'Reaction'

print(df_pi)
# optionally save
#df_pi.to_csv('fluxes_through_Pi_reactions.csv')


In [ ]:
qGluc = solution.fluxes['EX_Mal4']
qPi = solution.fluxes['EX_Pi']
qPPi = solution.fluxes['EX_PPi']
qAce = solution.fluxes['EX_Ace']
qCO2 = solution.fluxes['EX_CO2']
qHB = solution.fluxes['EX_PHB']

rates_df = pd.DataFrame([{
    'qGluc': qGluc,
    'qAce': qAce,
    'qPi': qPi,
    'qPPi': qPPi,
    'qCO2': qCO2,
    'qHB': qHB
}])
rates_df.to_csv('q_rates_labeling.csv', index=False)

import re

# Build flux_df without exchange reactions ---
non_ex_mask = [not rxn.startswith("EX_") for rxn in solution.fluxes.index]
flux_non_ex = solution.fluxes[non_ex_mask]

def standardize_arrow(formula: str) -> str:
    # Normalize any arrow (->, -->, <=>) into '<=>'
    return re.sub(r"\s*(?:<=>|--?>)\s*", " <=> ", formula)

# Build the DataFrame, applying standardize_arrow()
flux_df = pd.DataFrame({
    'Reaction Name': flux_non_ex.index,
    'Reaction Formula': [
        standardize_arrow(
            model.reactions.get_by_id(rxn).build_reaction_string()
        )
        for rxn in flux_non_ex.index
    ],
    'Relative Flux': flux_non_ex.values
})

flux_df.to_csv('fluxes_labeling_for_MDF.csv', index=False)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sympy import symbols, Eq, solve

INPUT_CSV = 'P3G_labeling_data.csv'

C13max = (1/6) * 100  # maximum 13C labeling
# Read the specific_rates.csv file
rates_df = pd.read_csv("specific_rates.csv")

# Extract and round qAc
qAc = round(rates_df.loc[rates_df["Variable"] == "qAc", "Value"].values[0], 2)
df = pd.read_csv(INPUT_CSV)
time_zero = df.loc[0, 't_experimental']
data = df.iloc[1:].reset_index(drop=True)

mol_P3G_NTS, mol_P3G_EMP, percent_EMP = symbols('mol_P3G_NTS mol_P3G_EMP percent_EMP')

def solve_flux(percent_13C):
    """
    Solve the system of 3 equations to compute percent via rubisco (NTS).
    Returns 100 - percent_EMP or np.nan on failure.
    """
    try:
        percent_rubisco = (percent_13C / C13max) * 100
        eq1 = Eq(mol_P3G_NTS / mol_P3G_EMP,
                 percent_rubisco / (100 - percent_rubisco))
        eq2 = Eq(percent_EMP / (100 - percent_EMP),
                 (mol_P3G_EMP * 3) / (mol_P3G_NTS * 2.5))
        eq3 = Eq(mol_P3G_NTS + 2*mol_P3G_EMP,
                 0.5 * ((-1*qAc) + mol_P3G_NTS + mol_P3G_EMP))
        sols = solve([eq1, eq2, eq3], [mol_P3G_NTS, mol_P3G_EMP, percent_EMP], dict=True)
        if not sols:
            return np.nan
        sol = sols[0]
        emp_value = float(sol[percent_EMP])
        if not np.isfinite(emp_value):
            return np.nan
        return 100 - emp_value
    except Exception:
        return np.nan

# mean
flux_mean = data['mean'].apply(solve_flux)
# mean + SD
flux_plus = (data['mean'] + data['SD']).apply(solve_flux)
# mean - SD
flux_minus = (data['mean'] - data['SD']).apply(solve_flux)

In [ ]:
OUTPUT_TIFF = 'flux_rubisco_labeling.tiff'
time = data['t_experimental'].values
flux = flux_mean.values
# error bars
err_low = np.clip(flux_mean - flux_minus, 0, None).values
err_up  = np.clip(flux_plus - flux_mean,  0, None).values

# Determine y-limits
valid = ~np.isnan(flux)
ylim_lower = np.nanmin(flux_minus[valid]) - 1
ylim_upper = np.nanmax(flux_plus[valid])  + 1

label_fontsize = 16    
tick_fontsize = 16    
inset_labelsize = 12
inset_ticksize = 12
legend_fontsize = 10

fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)

ax.errorbar(time, flux, yerr=[err_low, err_up], fmt='o',
            color='green', ecolor='gray', capsize=4, label='Data')
ax.plot(time_zero, 0, 'go', label='0,0')

ax.set_xlim(-1, 140)
ax.set_ylim(ylim_lower, ylim_upper)
ax.set_xlabel('time (min)', fontsize=label_fontsize)
ax.set_ylabel('glycolytic flux through rubisco (%)', fontsize=label_fontsize)
ax.tick_params(axis='both', labelsize=tick_fontsize)

inset_pos = [0.6, 0.25, 0.35, 0.35]
axins = fig.add_axes(inset_pos)
axins.errorbar(time, flux, yerr=[err_low, err_up], fmt='o',
               color='green', ecolor='gray', capsize=3)
axins.plot(time_zero, 0, 'go')
axins.axhline(y=y_baseline, linestyle='--', color='#e8690c', linewidth=2)
axins.set_xlim(-1, 22)
axins.set_ylim(ylim_lower, 16)
axins.tick_params(axis='both', labelsize=inset_ticksize)

# Save and show
plt.savefig(OUTPUT_TIFF, dpi=200)
plt.show()